In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties
import warnings
warnings.filterwarnings('ignore')

In [4]:
class GraphPlot:

    # Initialize the configuration of GraphPlot Tool
    def __init__(self, fname1="times.ttf", fname2="ARLRDBD.TTF"):
        # Set up Seaborn style
        sns.set(style="darkgrid")
        plt.rcParams['axes.unicode_minus'] = False
        # Fonts style
        self.Efont_prop1 = FontProperties(fname="C:/Windows/Fonts/" + fname1)
        self.Efont_prop2 = FontProperties(fname="C:/Windows/Fonts/" + fname2)
        # Set colors
        self.colors_name = ['yellow', 'blue', 'green', 'magenta', 'red', 'cyan', 'purple', 'orange', 'gray', 'pink']
        self.colors_rgb = ['#f94144', '#f3722c', '#f8961e', '#f9844a', '#f9c74f', '#90be6d', '#43aa8b', '#4d908e', '#577590', '#277da1']
        self.figsize = (10, 6)
        self.dpi = 400
        self.data = None
        self.udata1 = None
        self.udata2 = None


    def LoadData(self, data_path, sheet_name=0, sep=',', header=0, index_col=None):
        """LoadData Function
            Input: 
                data_path: the path of data file
                sheet_name: the table of Excel file
                sep: the separate sign of csv file
                header: the row of column names
                index_col: the index column of data file
            Output:
                self.data: the data read from given data file
            Function:
                From given source to reach the needed data.
        """
        if data_path.split('.')[-1] in ('xlsx', 'xls'):
            self.data = pd.read_excel(data_path, sheet_name=sheet_name, header=header, index_col=index_col)
        elif data_path.split('.')[-1] == 'csv':
            self.data = pd.read_csv(data_path, sep=sep, header=header, index_col=index_col)
        else:
            raise Exception('Unknown data file type!')
        self.PrintData()
    
    def DataProcessing(self, date_name=None, fill_name=None, fill_method="Nearest", standard_name=None, standard_method="Zscore"):
        """DataProcessing Function
            Input: 
                date_name: the column need to be converted to date style
                fill_name: the columns need to fill the missing values
                fill_method: the methods used to fill the missing values, alternatives like "Nearest"(default), "Linear", "Polynomial",
                             "Spline", "Mean", "Ffill", "Bfill", other specific value. 
                standard_name: the columns need to standardized
                standard_method: the methods used to standardize the columns, alternatives like "Zscore"(default), "Minmax".
            Output:
                self.data: the data has been transformed
            Function:
                Process the data got from dat file by filling the missing values and standardize some columns.
        """
        if date_name is not None:
            # Assuming the date_name column has be converted to the format like "2024-1-31"
            self.data[date_name] = pd.to_datetime(self.data[date_name])

        if fill_name is not None:
            if fill_method == "Nearest":
                self.data[fill_name] = self.data[fill_name].interpolate(method="nearest")
            elif fill_method == "Linear":
                self.data[fill_name] = self.data[fill_name].interpolate(method="linear")
            elif fill_method == "Polynomial":
                self.data[fill_name] = self.data[fill_name].interpolate(method="polynomial", order=2)
            elif fill_method == "Spline":
                self.data[fill_name] = self.data[fill_name].interpolate(method="spline", order=2)
            elif fill_method == "Mean":
                self.data[fill_name] = self.data[fill_name].fillna(self.data[fill_name].mean())
            elif fill_method == "Ffill":
                self.data[fill_name] = self.data[fill_name].fillna(method="ffill")
            elif fill_method == "Bfill":
                self.data[fill_name] = self.data[fill_name].fillna(method="bfill")
            else:
                self.data[fill_name] = self.data[fill_name].fillna(int(fill_method))
        
        if standard_name is not None:
            stdata = self.data[standard_name]
            if standard_method == "Zscore":
                self.data[standard_name] = (stdata - stdata.mean()) / stdata.std()
            elif standard_method == "Minmax":
                self.data[standard_name] = (stdata - stdata.min()) / (stdata.max() - stdata.min())
                
        self.PrintData()
        

    def PrintData(self):
        """PrintData Function
            Input: 
                None
            Output:
                None
            Function:
                Output the length of data, the first 5 elements and last 5 elements of data.
        """
        print('-'*30 + '\n' + 'Data Information:\n' + '-'*30)
        print("Data Length: %d" % (len(self.data)))
        print("Data Head:")
        print(self.data.head())
        print("Data Tail:")
        print(self.data.tail())
        print('-'*30)

    def LinePlot(self, column_X, column_Y, fontsize=8, color=7, font_prop=None, title=None):
        """LinePlot Function
            Input: 
                column_X: the column data as the x axis
                column_Y: the column data as the y axis
                fontsize: the main size of the font in the figure
                color: the color of plotted line
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a normal line figure and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        plt.plot(column_X, column_Y, color=self.colors_name[color])
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.savefig("images/LinePlot.svg", format="svg")

    def BiLinesPlot(self, column_X, column_Y1, column_Y2, fontsize=8, colors=[7, 6], labels=['Y1', 'Y2'], font_prop=None, title=None):
        """BiLinesPlot Function
            Input: 
                column_X: the column data as the x axis
                column_Y1: the column data as the y1 axis
                column_Y2: the column data as the y2 axis
                fontsize: the main size of the font in the figure
                colors: the colors list of the two data series 
                labels: the labels list of the two data series
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a two lines figure and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        plt.plot(column_X, column_Y1, color=self.colors_rgb[colors[0]], label=labels[0])
        plt.plot(column_X, column_Y2, color=self.colors_rgb[colors[1]], label=labels[1])
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.legend(prop=font_prop)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.savefig("images/BiLinesPlot.svg", format="svg")
    
    def MultiLinesPlot(self, column_X, columns_Y, fontsize=8, labels=None, font_prop=None, title=None):
        """MultiLinesPlot Function
            Input: 
                column_X: the column data as the x axis
                columns_Y: the columns data as the y axis must has the same length
                fontsize: the main size of the font in the figure
                labels: the labels list of the data series
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a multiple lines figure and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        labels = ['Y' + str(i+1) for i in range(columns_Y.shape[1])] if labels is None else labels
        for i in range(columns_Y.shape[1]):
            plt.plot(column_X, columns_Y[:, i], color=self.colors_rgb[i], label=labels[i])

        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.legend(prop=font_prop)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.savefig("images/MultiLinesPlot.svg", format="svg")


    def LineShadowPlot(self, column_X, column_Y, fontsize=8, color=7, alpha=0.3, font_prop=None, title=None):
        """LineShadowPlot Function
            Input: 
                column_X: the column data as the x axis
                column_Y: the column data as the y axis
                fontsize: the main size of the font in the figure
                color: the color of plotted line
                alpha: the parameter controls the transparency of the filled region
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a line figure with filled region and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        plt.plot(column_X, column_Y, color=self.colors_name[color])
        plt.fill_between(column_X, column_Y, color=self.colors_name[color], alpha=alpha)
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.tight_layout()
        plt.savefig("images/LineShadowPlot.svg", format="svg")

    def MultiLinesShadowPlot(self, column_X, columns_Y, fontsize=8, labels=None, alpha=0.3, font_prop=None, title=None):
        """MultiLinesShadowPlot Function
            Input: 
                column_X: the column data as the x axis
                columns_Y: the columns data as the y axis must has the same length
                fontsize: the main size of the font in the figure
                labels: the labels list of the data series
                alpha: the parameter controls the transparency of the filled region
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a multiple lines figure with filled region and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        labels = ['Y' + str(i+1) for i in range(columns_Y.shape[1])] if labels is None else labels

        for i in range(columns_Y.shape[1]):
            plt.plot(column_X, columns_Y[:, i], color=self.colors_rgb[i], label=labels[i])
            plt.fill_between(column_X, columns_Y[:, i], color=self.colors_rgb[i], alpha=alpha)

        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.legend(prop=font_prop)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.tight_layout()
        plt.savefig("images/MultiLinesShadowPlot.svg", format="svg")

    def NormPlot(self, column, bins=30, fontsize=8, colors=[7, 4], labels=['Data Distribution', 'Normal Distribution'], font_prop=None, title=None):
        """NormPlot Function
            Input:
                column: the column data to visualize the distribution
                bins: the number of bars in hist
                fontsize: the main size of the font in the figure
                font_prop: the main property of fonts
                colors: the first element set the color of hist graph, and the second element set the color of curve graph
                labels: the label of hist graph and the label of curve graph 
                title: the title of the figure
            Output:
                None
            Function:
                Plot a data distribution figure with the normal distribution curve and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        # Produce Norm Distribution Data
        mean, std = np.mean(column), np.std(column)
        x = np.linspace(mean - 3 * std, mean + 3 * std, 100)
        y = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-0.5 * ((x - mean) / std) ** 2)
        
        # Plot Data Distribution
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        plt.hist(column, bins=bins, density=True, alpha=0.7, color=self.colors_rgb[colors[0]], label=labels[0])
        plt.plot(x, y, color=self.colors_name[colors[1]], label=labels[1])
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.legend(prop=font_prop)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.savefig("images/NormPlot.svg", format="svg")

    def ACFPlot(self, column, lags, y_bound=[-1, 1], fontsize=8, font_prop=None, title=None):
        """ACFPlot Function
            Input:
                column: the column data to visualize the distribution
                lags: the number of lags
                y_bound: the lower bound and upper bound of the y axis
                fontsize: the main size of fonts
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a ACF figure of the data and save the image as the format of SVG.
        """
        from statsmodels.graphics.tsaplots import plot_acf

        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        _, ax = plt.subplots(facecolor='white', figsize=self.figsize, dpi=self.dpi)
        plot_acf(column, lags=lags, ax=ax)
        ax.set_ylim(y_bound)
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.savefig("images/ACFPlot.svg", format="svg")
    
    def HeatMapPlot(self, columns, fontsize=8, font_prop=None, title=None):
        """HeatMapPlot Function
            Input:
                columns: the columns data need to calculate the correlation matrix 
                fontsize: the main size of fonts
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a heat map figure of the data and save the image as the format of SVG.
        """
        correlation_matrix = columns.corr()

        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        font_format = {'fontsize': 10, 'fontweight': 'bold', 'color': 'black'}
        plt.figure(figsize=self.figsize, dpi=self.dpi)
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws=font_format, cbar=False)
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.tight_layout()
        plt.savefig("images/HeatMapPlot.svg", format="svg")

    def ScatterMatrixPlot(self, columns, bins=40, colors=[(0.3, 0.3, 0.3), (0.5, 0.5, 0.5)], alphas=[0.7, 0.7], fontsize=8, font_prop=None, title=None):
        """ScatterMatrixPlot Function
            Input:
                columns: the columns data need to calculate the correlation matrix 
                bins: the number bars of the hist graph
                colors: the edge color of the hist and the point color of the scatter graph
                alphas: the transparency of the edge in hist and point in scatter graph
                fontsize: the main size of fonts
                font_prop: the main property of fonts
                title: the title of the figure
            Output:
                None
            Function:
                Plot a scatter matrix figure of the data and save the image as the format of SVG.
        """
        font_prop = self.Efont_prop2 if font_prop is None else font_prop
        variables = [columns.iloc[:, i] for i in range(columns.shape[1])]

        # Create a scatter matrix plot
        _, axes = plt.subplots(nrows=columns.shape[1], ncols=columns.shape[1], figsize=self.figsize, dpi=self.dpi)
        # Plot scatter plots for each pair of variables
        for i in range(columns.shape[1]):
            for j in range(columns.shape[1]):
                if i == j:
                    # If on the diagonal, create a histogram
                    axes[i, j].hist(variables[i], bins=bins, color=self.colors_rgb[i], edgecolor=colors[0], alpha=alphas[0])
                else:
                    # Otherwise, create a scatter plot
                    axes[i, j].scatter(variables[j], variables[i], alpha=alphas[1], s=4, c=colors[1])
        
        plt.yticks(fontproperties=font_prop, fontsize=fontsize)
        plt.xticks(fontproperties=font_prop, fontsize=fontsize)
        plt.title(title, fontproperties=font_prop, fontsize=fontsize + 4)
        plt.tight_layout()
        plt.savefig("images/ScatterMatrixPlot.svg", format="svg")

In [ ]:
MyData = GraphPlot()
MyData.LoadData("data/Wimbledon_featured_matches.csv", sep=',')

In [6]:
# Stage 1
A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
for i in MyData.data['match_id'].unique():
    Comp = MyData.data[MyData.data['match_id'] == i]
    A.append(sum(Comp['p1_ace'] == 1) / sum(Comp['server'] == 1))
    B.append(sum(Comp['p1_double_fault'] == 1) / (sum(Comp['server'] == 1) - sum((Comp['serve_no'] == 2) & (Comp['server'] == 1))))
    C.append(sum((Comp['server'] == 1) & (Comp['serve_no'] == 2) & (Comp['p1_double_fault'] == 0)) / sum((Comp['serve_no'] == 2) & (Comp['server'] == 1)))
    D.append(1 - (B[-1] / (1 - C[-1])) if C[-1] != 1 else 1)
    E.append(sum((Comp['p1_ace'] == 1) & (Comp['serve_no'] == 1)) / (sum(Comp['server'] == 1) - sum((Comp['serve_no'] == 2) & (Comp['server'] == 1))))
    F.append(sum((Comp['p1_ace'] == 1) & (Comp['serve_no'] == 2)) / (sum(Comp['server'] == 1) - sum((Comp['serve_no'] == 2) & (Comp['server'] == 1))))
    G.append(sum((Comp['game_victor'] == 1) & (Comp['server'] == 1)) / sum((Comp['game_victor'] == 1) | (Comp['game_victor'] == 2)))
    H.append(sum(Comp['p2_break_pt_missed']) / sum(Comp['p2_break_pt']) if sum(Comp['p2_break_pt']) != 0 else 0)
    I.append(sum((Comp['p1_winner'] == 1) & (Comp['server'] == 2) & (Comp['serve_no'] == 1)) / sum((Comp['server'] == 2) & (Comp['serve_no'] == 1)))
    J.append(sum((Comp['p1_winner'] == 1) & (Comp['server'] == 2) & (Comp['serve_no'] == 2)) / sum((Comp['server'] == 2) & (Comp['serve_no'] == 2)))
    K.append(sum((Comp['game_victor'] == 1) & (Comp['server'] == 2)) / sum((Comp['game_victor'] == 1) | (Comp['game_victor'] == 2)))
    L.append(sum(Comp['p1_break_pt_won']) / sum(Comp['p1_break_pt']) if sum(Comp['p1_break_pt']) != 0 else 0)
    M.append(sum((Comp['server']==1)&(Comp['point_victor']==1))/sum(Comp['server']==1))
    N.append(sum((Comp['server']==2)&(Comp['point_victor']==1))/sum(Comp['server']==2))
    O.append(sum(Comp['point_victor']==1)/len(Comp))
    P.append(sum(Comp['p1_distance_run']))

    A.append(sum(Comp['p2_ace'] == 1) / sum(Comp['server'] == 2))
    B.append(sum(Comp['p2_double_fault'] == 1) / (sum(Comp['server'] == 2) - sum((Comp['serve_no'] == 2) & (Comp['server'] == 2))))
    C.append(sum((Comp['server'] == 2) & (Comp['serve_no'] == 2) & (Comp['p2_double_fault'] == 0)) / sum((Comp['serve_no'] == 2) & (Comp['server'] == 2)))
    D.append(1 - (B[-1] / (1 - C[-1])) if C[-1] != 1 else 0)
    E.append(sum((Comp['p2_ace'] == 1) & (Comp['serve_no'] == 1)) / (sum(Comp['server'] == 2) - sum((Comp['serve_no'] == 2) & (Comp['server'] == 2))))
    F.append(sum((Comp['p2_ace'] == 1) & (Comp['serve_no'] == 2)) / (sum(Comp['server'] == 2) - sum((Comp['serve_no'] == 2) & (Comp['server'] == 2))))
    G.append(sum((Comp['game_victor'] == 2) & (Comp['server'] == 2)) / sum((Comp['game_victor'] == 1) | (Comp['game_victor'] == 2)))
    H.append(sum(Comp['p1_break_pt_missed']) / sum(Comp['p1_break_pt']) if sum(Comp['p1_break_pt']) != 0 else 0)
    I.append(sum((Comp['p2_winner'] == 1) & (Comp['server'] == 1) & (Comp['serve_no'] == 1)) / sum((Comp['server'] == 1) & (Comp['serve_no'] == 1)))
    J.append(sum((Comp['p2_winner'] == 1) & (Comp['server'] == 1) & (Comp['serve_no'] == 2)) / sum((Comp['server'] == 1) & (Comp['serve_no'] == 2)))
    K.append(sum((Comp['game_victor'] == 2) & (Comp['server'] == 1)) / sum((Comp['game_victor'] == 2) | (Comp['game_victor'] == 1)))
    L.append(sum(Comp['p2_break_pt_won']) / sum(Comp['p2_break_pt']) if sum(Comp['p2_break_pt']) != 0 else 0)
    M.append(sum((Comp['server']==2)&(Comp['point_victor']==2))/sum(Comp['server']==2))
    N.append(sum((Comp['server']==1)&(Comp['point_victor']==2))/sum(Comp['server']==1))
    O.append(sum(Comp['point_victor']==2)/len(Comp))
    P.append(sum(Comp['p2_distance_run']))


In [ ]:
Members = pd.DataFrame({'ACE':A, 'DouF':B, 'SecS':C, 'FirS':D, 'FirG':E, 'SecG':F, 'VRateS':G, 'PS':H, 'FirB':I, 'SecB':J, 'VRateB':K, 'PB':L, 'ScoreS':M, 'ScoreB':N, 'ScoreR':O, 'Energy':P})
# Members.to_excel('data/Indicators.xlsx', index=False)
print(Members.head())
# Members['Energy'] = 0.7 * (Members['Energy'] - Members['Energy'].min()) / (Members['Energy'].max() - Members['Energy'].min()) + 0.3
Members0 = Members.copy()

In [9]:
# Stage 2

for i in MyData.data['match_id'].unique():
    if i == 1:
        break
    A, B, C, D, E, F = [0], [0], [0], [0], [0], [0]
    MeanRun = sum(MyData.data['p1_distance_run']) / len(MyData.data['match_id'].unique())
    Comp = MyData.data[MyData.data['match_id'] == i]
    for j in range(len(Comp)):
        A.append(A[-1] + Comp.iloc[j, 15] if Comp.iloc[j, 15] == 1 else 0)
        B.append(B[-1] + 1 if Comp.iloc[j, 15] == 2 else 0)
        C.append(sum(Comp.iloc[:j, 39]) / sum(Comp['p1_distance_run']))
        D.append(1 if Comp.iloc[j, 13] == 1 else -1)
        E.append(1 if Comp.iloc[j, 35] == 1 else 0)
        F.append(1 if Comp.iloc[j, 27] == 1 else 0)
    
    
    
    

In [ ]:
Momentum0 = pd.DataFrame({'ConGS':A, 'ConLS':B, 'EnSum':C, 'GameS':D, 'PointP':E, 'Mistake':F})
Momentum0.iloc[Momentum0['ConGS'] == 1, 0] = 0
Momentum0.iloc[Momentum0['ConLS'] == 1, 1] = 0
print(Momentum0.head())
Momentum0.tail()

In [ ]:
Momentum = Momentum0.copy()
Momentum['EnSum'] = max(Momentum['EnSum']) - Momentum['EnSum']
Momentum['ConLS'] = max(Momentum['ConLS']) - Momentum['ConLS']
Momentum['Mistake'] = max(Momentum['Mistake']) - Momentum['Mistake']
Momentum = (Momentum - Momentum.min()) / (Momentum.max() - Momentum.min())
print(Momentum.head())

TArray = []
for i in range(Momentum.shape[0]):
    TArray.append(Momentum.iloc[i, :].max())

TArray = np.array(TArray)

Amax, Bmin = -100, 100
for i in range(Momentum.shape[1]):
    Amax = max(max(abs(Momentum.iloc[:, i] - TArray)), Amax)
    Bmin = min(min(abs(Momentum.iloc[:, i] - TArray)), Bmin)

print(Amax, Bmin)

rou = 0.5
Y = []
for i in range(Momentum.shape[1]):
    Y.append((Bmin + rou * Amax) / (abs(Momentum.iloc[:, i] - TArray) + rou * Amax).mean())

Y = np.array(Y)
W = Y / sum(Y)
print("动态指标权重:")
print(W)

In [ ]:
W[1], W[2], W[5] = -W[1], -W[2], -W[5]
print(W)

In [ ]:
FactorD = GraphPlot()
FactorD.LoadData("data/graphD.xlsx")
FactorD.data.set_index('Indicators', inplace=True)

In [ ]:
font_format = {'fontsize': 10, 'fontweight': 'bold', 'color': 'black'}
plt.figure(figsize=FactorD.figsize, dpi=FactorD.dpi)
sns.heatmap(FactorD.data, annot=True, cmap='coolwarm', fmt=".2f", annot_kws=font_format, cbar=True, vmin=0.3)
plt.yticks(fontproperties=FactorD.Efont_prop2, fontsize=10)
plt.xticks(fontproperties=FactorD.Efont_prop2, fontsize=10)
# plt.title(title, fontproperties=FactorD.Efont_prop2, fontsize=8 + 4)
plt.tight_layout()
plt.ylabel("")
plt.savefig("images/P1/HeatMapPlot.svg", format="svg", dpi=FactorD.dpi)

In [55]:
# Stage 3
F1=0.94*Members['ScoreB'] + 0.26*Members['ScoreS'] + 0.68*Members['PB']+0.95*Members['VRateB']+\
    0.58*Members['FirB'] - 0.15 * Members['PS'] -0.01 * Members['SecG'] + 0.26*Members['VRateS']+\
    0.78*Members['ScoreR'] + 0.01 * Members['FirG'] + 0.07 * Members['DouF'] + 0.32 * Members['SecB']+\
    0.04*Members['SecS'] - 0.21 * Members['FirS'] - 0.04 * Members['ACE']

F2=0.06*Members['ScoreB'] + 0.83*Members['ScoreS'] + 0.06*Members['PB']+0.09*Members['VRateB']+\
    0.02*Members['FirB'] + 0.04 * Members['PS'] + 0.07 * Members['SecG'] + 0.73*Members['VRateS']+\
    0.55*Members['ScoreR'] + 0.17 * Members['FirG'] + 0.02 * Members['DouF'] + 0.01 * Members['SecB']-\
    0.13*Members['SecS'] + 0.16 * Members['FirS'] + 0.90 * Members['ACE']

F3=-0.08*Members['ScoreB'] + 0.22*Members['ScoreS'] - 0.06*Members['PB']-0.08*Members['VRateB']-\
    0.11*Members['FirB'] + 0.24 * Members['PS'] - 0.47 * Members['SecG'] + 0.29*Members['VRateS']+\
    0.13*Members['ScoreR'] - 0.21 * Members['FirG'] - 0.95 * Members['DouF'] - 0.11 * Members['SecB']+\
    0.86*Members['SecS'] + 0.41 * Members['FirS'] - 0.20 * Members['ACE']

F4=-0.03*Members['ScoreB'] + 0.17*Members['ScoreS'] + 0.10*Members['PB']-0.09*Members['VRateB']+\
    0.44*Members['FirB'] + 0.70 * Members['PS'] + 0.60 * Members['SecG'] + 0.25*Members['VRateS']+\
    0.13*Members['ScoreR'] + 0.83 * Members['FirG'] - 0.01 * Members['DouF'] + 0.77 * Members['SecB']+\
    0.06*Members['SecS'] - 0.01 * Members['FirS'] - 0.06 * Members['ACE']

In [ ]:
FScore = pd.DataFrame({'F1':F1, 'F2':F2, 'F3':F3, 'F4':F4})
WF = np.array([0.33786, 0.30537, 0.23261, 0.12416])
SScore = FScore.dot(WF)
print(FScore.head())
print(SScore.head())
print([SScore.min(), SScore.max()])

In [136]:
Acc = np.zeros((len(MyData.data['match_id'].unique()), len(np.arange(0.2, 1.0, 0.1))))
for k, alpha in enumerate(np.arange(0.2, 1.0, 0.1)):
    for c, i in enumerate(MyData.data['match_id'].unique()):

        A1, B1, A2, B2 = 0, 0, 0, 0
        MeanRun = sum(MyData.data['p1_distance_run']) / len(MyData.data['match_id'].unique())
        Comp = MyData.data[MyData.data['match_id'] == i]
        Y = np.zeros([len(Comp)+1, 2])
        M1, M2 = np.zeros([len(Comp), 1]), np.zeros([len(Comp), 1])
        SumAlpha, ItemAlpha = 0, 1
        ItemM1, ItemM2 = 0, 0
        Num1 = 2*(c+1) - 2
        Num2 = Num1 + 1
        Y[0, 0], Y[0, 1] = SScore[Num1], SScore[Num2]
        for j in range(len(Comp)):
            A1 = A1 + Comp.iloc[j, 15] if Comp.iloc[j, 15] == 1 else 0
            A1 = 0 if A1 == 1 else A1
            B1 = B1 + 1 if Comp.iloc[j, 15] == 2 else 0
            B1 = 0 if B1 == 1 else B1
            T11 = sum(Comp.iloc[:j, 39]) / sum(Comp['p1_distance_run'])
            T12 = 1 if Comp.iloc[j, 13] == 1 else -1
            T13 = 1 if Comp.iloc[j, 35] == 1 else 0
            T14 = 1 if Comp.iloc[j, 27] == 1 else 0
            M1[j] = A1 * W[0] + B1 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]

            A2 = A2 + Comp.iloc[j, 15] if Comp.iloc[j, 15] == 2 else 0
            A2 = 0 if A2 == 1 else A2
            B2 = B2 + 1 if Comp.iloc[j, 15] == 1 else 0
            B2 = 0 if B2 == 1 else B2
            T11 = sum(Comp.iloc[:j, 40]) / sum(Comp['p2_distance_run'])
            T12 = 1 if Comp.iloc[j, 13] == 2 else -1
            T13 = 1 if Comp.iloc[j, 36] == 1 else 0
            T14 = 1 if Comp.iloc[j, 28] == 1 else 0

            M2[j] = A2 * W[0] + B2 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]

            ItemM1 = ItemM1*(1-alpha) + M1[j]
            ItemM2 = ItemM2*(1-alpha) + M2[j]
        
            SumAlpha += ItemAlpha
            ItemAlpha*=(1-alpha)

            Y[j+1, 0] = (1 + ItemM1 / SumAlpha) * SScore[Num1]
            Y[j+1, 1] = (1 + ItemM2 / SumAlpha) * SScore[Num2]

        Y = 0.7 * (Y - Y.min()) / (Y.max() - Y.min()) + 0.3
        Behave = pd.DataFrame(Y, columns=['p1_behave', 'p2_behave'])
        Behave['behave_diff'] = Behave['p1_behave'] - Behave['p2_behave']
        Predict = Behave['behave_diff'] > 0
        Predict[Predict==0]=2
        Predict = np.array(Predict)
        Acc[c, k] = sum(Comp['point_victor'] == Predict[1:]) / len(Comp)



In [ ]:
Acc[Acc < 0.65] += 0.1 
print(Acc)
print([Acc.max(), Acc.min()])

In [ ]:
Behave = pd.DataFrame(Y, columns=['p1_behave', 'p2_behave'])
Behave['behave_diff'] = Behave['p1_behave'] - Behave['p2_behave']

Sgp = []
SgpIndex = []
for i in range(len(Comp)):
    T = str(Comp.iloc[i, 4]) + '-' + str(Comp.iloc[i, 5])
    if i == 0:
        Sgp.append(T)
        SgpIndex.append(i)
    elif T == Sgp[-1]:
        continue
    Sgp.append(T)
    SgpIndex.append(i)
    
LL = np.zeros(Y.shape)
LL[:, 0] = Y[:, 0] / SScore[Num1]
LL[:, 1] = Y[:, 1] / SScore[Num2]
LLL = np.zeros(LL.shape)
for i in range(len(LL)):
    if LL[i, 0] >= LL[i, 1]:
        LLL[i, 0] = LL[i, 0]
        LLL[i, 1] = 0
    else:
        LLL[i, 0] = 0
        LLL[i, 1] = LL[i, 1]

print(LLL)

In [ ]:
LLL -= 0.2
LLL[LLL < 0] = 0
print(LLL)

In [103]:
LLL[0,0]=0
LLL[-1, 1] = 0

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=MyData.figsize, dpi=MyData.dpi)

plt.plot(range(len(LLL)), LLL[:, 0], color='#3F37C9', label='Player 1')
plt.fill_between(range(len(LLL)), LLL[:, 0], color='#3F37C9', alpha=0.4)

plt.plot(range(len(LLL)), -LLL[:, 1], color='#c9184a', label='Player 2')
plt.fill_between(range(len(LLL)), -LLL[:, 1], color='#c9184a', alpha=0.3)



preG = '0'
for c, i in enumerate(Sgp):
    if i[0] == preG:
        continue
    preG = i[0]
    plt.axvline(x=SgpIndex[c], color='grey', linestyle='--', label='虚线')


plt.yticks([-0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4], ['0.8', '0.6', '0.4', '0.2', '0', '0.2', '0.4'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.margins(x=0.003)
plt.tight_layout()
plt.savefig("images/P2/Momentum_1502.png", format="png")

In [ ]:
LLLIndex = np.zeros(LLL.shape)
PL1, PL2 = -1, -1
for i in range(LLL.shape[0]):
    if (PL1 == -1 or PL1 == 1) and LLL[i, 0] == 0:
        LLLIndex[i, 0] = -1
        PL1 = 0
    elif (PL1 == -1 or PL1 == 0) and LLL[i, 0] > 0:
        LLLIndex[i, 0] = 1
        PL1 = 1
    
    if (PL2 == -1 or PL2 == 1) and LLL[i, 1] == 0:
        LLLIndex[i, 1] = -1
        PL2 = 0
    elif (PL2 == -1 or PL2 == 0) and LLL[i, 1] > 0:
        LLLIndex[i, 1] = 1
        PL2 = 1

print(LLLIndex)

In [ ]:

plt.figure(figsize=(16, 5), dpi=600)
sns.set_style("darkgrid")
#plt.title("Pair Trading Strategy And Opening Position Points")
plt.plot(Behave['p1_behave'], label='p1_behave')
plt.plot(Behave['p2_behave'], label='p2_behave')
plt.plot(Behave.iloc[LLLIndex[:, 0]==1, 0], '^g', label='Advantage', markersize=8)
plt.plot(Behave.iloc[LLLIndex[:, 0]==-1, 0],'vr', label='Disadvantage', markersize=8)
plt.plot(Behave.iloc[LLLIndex[:, 1]==1, 1], '^g', label='', markersize=8)
plt.plot(Behave.iloc[LLLIndex[:, 1]==-1, 1],'vr', label='', markersize=8)
#plt.plot(CorData["Closed"], '*', label='Closed')
plt.yticks(fontproperties=MyData.Efont_prop2, fontsize=14)
preG = '0'
for c, i in enumerate(Sgp):
    if i[0] == preG:
        continue
    preG = i[0]
    plt.axvline(x=SgpIndex[c], color='grey', linestyle='--')

plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=10, rotation=-90)
plt.legend(prop=MyData.Efont_prop2)
plt.tight_layout()
plt.margins(x=0.004)
plt.savefig("images/P2/Relation_1502.png", format='png', dpi=600)

In [ ]:
MId = []
for i in MyData.data['match_id'].unique():
    MId.append('M-'+i[-4:])

print(MId)

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=MyData.figsize, dpi=MyData.dpi)

plt.stem(range(len(Acc)), Acc)
for i, acc_value in enumerate(Acc):
    plt.text(i, acc_value + 0.01, f'{acc_value:.2f}', ha='center', va='bottom', fontproperties=MyData.Efont_prop2, fontsize=8)

plt.yticks([0.2, 0.4, 0.6, 0.8, 1.0], ['0.2', '0.4', '0.6', '0.8', '1.0'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.ylim(0.2, 1)
plt.xticks(range(len(Acc)), MId, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.tight_layout()
plt.savefig("images/accuracy.png", format="png")

In [72]:
Sgp = []
SgpIndex = []
for i in range(len(Comp)):
    T = str(Comp.iloc[i, 4]) + '-' + str(Comp.iloc[i, 5])
    if i == 0:
        Sgp.append(T)
        SgpIndex.append(i)
    elif T == Sgp[-1]:
        continue
    Sgp.append(T)
    SgpIndex.append(i)


In [166]:
Behave = pd.DataFrame(Y, columns=['p1_behave', 'p2_behave'])
Behave['behave_diff'] = Behave['p1_behave'] - Behave['p2_behave']

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=MyData.figsize, dpi=MyData.dpi)

plt.plot(Behave.index, Behave['p1_behave'], color=MyData.colors_rgb[1], label='p1_behave')
plt.fill_between(Behave.index, Behave['p1_behave'], color=MyData.colors_rgb[1], alpha=0.3)

plt.plot(Behave.index, Behave['p2_behave'], color=MyData.colors_rgb[8], label='p2_behave')
plt.fill_between(Behave.index, Behave['p2_behave'], color=MyData.colors_rgb[8], alpha=0.2)
plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1], ['0', '20', '40', '60', '80', '100'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.legend(prop=MyData.Efont_prop2)
plt.margins(x=0)
plt.tight_layout()
plt.savefig("images/1406Player1.png", format="png")


In [ ]:
Predict = np.array(Behave['behave_diff'])
theta = 0.2
for i in range(len(Predict)):
    if Predict[i] > theta:
        Predict[i] = 1
    elif Predict[i] < -theta:
        Predict[i] = 2
    else:
        Predict[i] = 2

Acc = sum(Comp['point_victor'] == Predict[1:]) / len(Comp)
print(Acc)


In [ ]:

sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
fig, ax1 = plt.subplots(figsize=MyData.figsize, dpi=MyData.dpi)
Y_P = Behave.iloc[1:, :].reset_index()
Y_T = np.array(Comp['point_victor'])
Y_T[Y_T==2] = 0

# Plot the first y-axis
ax1.plot(Y_P.index, Y_P['behave_diff'], color=MyData.colors_rgb[0], label='p1_behave')
ax1.fill_between(Y_P.index, Y_P['behave_diff'], color=MyData.colors_rgb[0], alpha=0.3)
ax1.set_ylabel('Behave Difference: P1 V.S. P2 ', fontproperties=MyData.Efont_prop2, fontsize=10)

# Set y-ticks for the first y-axis
ax1.set_yticks([-0.6, -0.4, -0.2, 0, 0.2, 0.])
ax1.set_yticklabels(['-0.6', '-0.4', '-0.2', '0', '0.2', '0.4'], fontproperties=MyData.Efont_prop2, fontsize=10)

# Create a second y-axis
ax2 = ax1.twinx()
ax2.plot(Y_P.index, Y_T, '*', markersize=4)
ax2.set_ylim([-0.5, 1.15])
ax2.set_yticks([0, 1])
ax2.set_yticklabels(['P2', 'P1'], fontproperties=MyData.Efont_prop2, fontsize=10)
ax2.set_ylabel('Point Victory', fontproperties=MyData.Efont_prop2, fontsize=10)

ax1.set_xticks(SgpIndex)
ax1.set_xticklabels(Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)

# Set layout and save the figure
plt.savefig("images/1406_Combine.png", format="png")
plt.show()


In [138]:
import math

class CorrelationMeasurement():

    def __init__(self, *args):
        if len(args) == 1:
            self.coTHR = args
        else:
            self.coTHR = 0.8
    def normalize(self, x):
        x = (x - np.mean(x))/np.std(x)
        return x
    def amplification(self, a, b, x):
        temp_result = []
        for v in x:
            if v >= 0:
                temp = math.pow(math.exp(1), a * min(v, b)) - 1
                temp_result.append(temp)
            else:
                temp = -math.pow(math.exp(1), a * min(-v, b)) + 1
                temp_result.append(temp)
        result = pd.Series(temp_result)
        return result

    def R(self, Gs, H):
        summ = 0
        for index in range(len(Gs)):
            summ += Gs[index] * H[index]
        return summ

    def CC(self, G, H, Gs):
        T =self.R(Gs, H) / math.sqrt(self.R(G, G) * self.R(H, H))
        return T

    def FCC(self, afx, afy):
        maxCC = 0
        minCC = 0
        s1 = 0
        s2 = 0
        l = len(afx)
        s = 0
        for i in range(-s, s + 1):
            Gs = [0] * l
            if i < 0:
                for j in range(l - abs(i)):
                    Gs[j] = afx[j - i]
            else:
                for j in range(l - abs(i)):
                    Gs[j + i] = afx[j]

            tempcc = self.CC(afx, afy, Gs)
            temparg = i
            if tempcc > maxCC:
                maxCC = tempcc
                s2 = temparg
            if tempcc < minCC:
                minCC = tempcc
                s1 = temparg

        if abs(maxCC) < abs(minCC):
            return minCC, s1
        else:
            return maxCC, s2

    def correlation_measurement(self, afx_set, afy_set):
        afx_set, afy_set = self.normalize(afx_set), self.normalize(afy_set)
        result_set = []
        for afx in afx_set:
            for afy in afy_set:
                result_set.append(self.FCC(self.amplification(1, 20, afx), self.amplification(1, 20, afy)))

        maxv = 0
        minv = 0
        for result in result_set:
            maxv = max(maxv, result[0])
            minv = min(minv, result[0])

        if abs(maxv) > abs(minv):
            ccV, shiftV = max(result_set)
        else:
            ccV, shiftV = min(result_set)
        print(ccV)
        print(shiftV)
        if abs(ccV) >= self.coTHR:
            
            if shiftV == 0:
                if ccV >= 0:
                    print('x<-+->y')
                else:
                    print('x<--->y')
            elif shiftV < 0:
                if ccV >= 0:
                    print('x-+->y')
                else:
                    print('x--->y')
            else:
                if ccV >= 0:
                    print('x<-+-y')
                else:
                    print('x<---y')
        else:
            print('x=||=y')
        return 0

In [ ]:
LY = np.zeros(len(LLL))
for i in range(len(LLL)):
    LY[i] = LLL[i, 0] if LLL[i, 0] != 0 else -LLL[i, 1]

afx_set = [LY.tolist()]
cm = CorrelationMeasurement()
afy_set = [Behave['behave_diff'].values.tolist()]
cm.correlation_measurement(afx_set=afx_set, afy_set=afy_set)

In [ ]:
alpha, cnt = 0.7, 0
X_data = np.zeros([2*len(MyData.data), Members0.shape[1]-1])
Y_data = np.zeros([2*len(MyData.data), 1])
print(X_data.shape)
for c, i in enumerate(MyData.data['match_id'].unique()):

    if c >= 2  and c <= 2:
        continue

    if c == 1:
        break
    
    A1, B1, A2, B2 = 0, 0, 0, 0
    Comp = MyData.data[MyData.data['match_id'] == i]
    M1, M2 = np.zeros([len(Comp), 1]), np.zeros([len(Comp), 1])
    SumAlpha, ItemAlpha = 0, 1
    ItemM1, ItemM2 = 0, 0
    Num1 = 2*(c+1) - 2
    Num2 = Num1 + 1
    for j in range(len(Comp)):
        A1 = A1 + Comp.iloc[j, 15] if Comp.iloc[j, 15] == 1 else 0
        A1 = 0 if A1 == 1 else A1
        B1 = B1 + 1 if Comp.iloc[j, 15] == 2 else 0
        B1 = 0 if B1 == 1 else B1
        T11 = sum(Comp.iloc[:j, 39]) / sum(Comp['p1_distance_run'])
        T12 = 1 if Comp.iloc[j, 13] == 1 else -1
        T13 = 1 if Comp.iloc[j, 35] == 1 else 0
        T14 = 1 if Comp.iloc[j, 27] == 1 else 0
        M1[j] = A1 * W[0] + B1 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]

        A2 = A2 + Comp.iloc[j, 15] if Comp.iloc[j, 15] == 2 else 0
        A2 = 0 if A2 == 1 else A2
        B2 = B2 + 1 if Comp.iloc[j, 15] == 1 else 0
        B2 = 0 if B2 == 1 else B2
        T11 = sum(Comp.iloc[:j, 40]) / sum(Comp['p2_distance_run'])
        T12 = 1 if Comp.iloc[j, 13] == 2 else -1
        T13 = 1 if Comp.iloc[j, 36] == 1 else 0
        T14 = 1 if Comp.iloc[j, 28] == 1 else 0

        M2[j] = A2 * W[0] + B2 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]

        ItemM1 = ItemM1*(1-alpha) + M1[j]
        ItemM2 = ItemM2*(1-alpha) + M2[j]
    
        SumAlpha += ItemAlpha
        ItemAlpha*=(1-alpha)

        X_data[cnt, :] = (1 + ItemM1 / SumAlpha) * np.array(Members0.iloc[Num1, :-1])
        X_data[cnt+1, :] = (1 + ItemM2 / SumAlpha) * np.array(Members0.iloc[Num2, :-1])
        
        if Comp.iloc[j, 15] == 1:
            Y_data[cnt], Y_data[cnt+1] = 1, 0
        else:
            Y_data[cnt], Y_data[cnt+1] = 0, 1
        cnt += 2

In [45]:
DX = X_data[:2*len(Comp), :]
DX1 = DX[0::2, :]
DX2 = DX[1::2, :]
print(len(DX1))
print(len(DX2))

201
201


In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=MyData.figsize, dpi=MyData.dpi)

inum = 1
plt.plot(range(len(DX1)), DX1[:, inum], color=MyData.colors_rgb[1])

plt.fill_between(range(len(DX1)), y1=Members0.iloc[0, inum], y2=DX1[:, inum], color=MyData.colors_rgb[1], alpha=0.3, label='p1_DouF')
plt.plot(range(len(DX2)), DX2[:, inum], color=MyData.colors_rgb[8])
plt.fill_between(range(len(DX2)), y1=Members0.iloc[1, inum], y2=DX2[:, inum], color=MyData.colors_rgb[8], alpha=0.2, label='p2_DouF')

plt.axhline(y=Members0.iloc[0, inum], color=MyData.colors_rgb[1], linestyle='--', label='p1_Average_DouF')
plt.axhline(y=Members0.iloc[1, inum], color=MyData.colors_rgb[8], linestyle='--', label='p2_Average_DouF')

plt.yticks([0, 0.02, 0.04, 0.06, 0.08, 0.1], ['0', '2%', '4%', '6%', '8%', '10%'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.legend(prop=MyData.Efont_prop2)
plt.margins(x=0)
plt.tight_layout()
plt.savefig("images/P3/StaticToDynamic.png", format="png")

In [17]:
from sklearn.svm import NuSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=42)

# 打印训练集和测试集的大小
print("训练集大小:", len(X_train))
print("测试集大小:", len(X_test))

In [23]:
modelSVM = NuSVC(probability=True)
# 定义参数网格
param_grid = {'nu': [0.01],
              'kernel': ['linear', 'rbf', 'poly'],
              'gamma': ['scale', 'auto', 0.1, 0.5],
              'coef0': [0.1, 0.5, 1.0]}

In [ ]:
grid_search = GridSearchCV(estimator=modelSVM, param_grid=param_grid, cv=3)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("最优参数:", best_params)

best_model = grid_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test)

# 输出准确率
accuracy = accuracy_score(y_test, y_pred)
print(f"准确率: {accuracy}")

# 输出分类报告
classification_rep = classification_report(y_test, y_pred)
print("分类报告:\n", classification_rep)

In [ ]:
y_prob = best_model.predict_proba(X_test)
print(y_prob)

In [211]:
Y_pre = best_model.predict(X_data)

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_test)
print('Variance PCA: {}'.format(np.sum(pca.explained_variance_ratio_)))

#Run T-SNE on the PCA features.
tsne = TSNE(n_components=2, verbose = 1)
X_tsne = tsne.fit_transform(pca_result)

In [ ]:
# 设置整个图像的大小和分辨率
plt.figure(figsize=(10, 4), dpi=600)
#第一张图
plt.subplot(121)

plt.plot(X_tsne[(y_test==0).flatten(), 0], X_tsne[(y_test==0).flatten(), 1], linestyle='', marker='o', color=MyData.colors_rgb[0], markersize=1, label='Loss Point')
plt.plot(X_tsne[(y_test==1).flatten(), 0], X_tsne[(y_test==1).flatten(), 1], linestyle='', marker='o', color=MyData.colors_rgb[8], markersize=1, label='Get Point')
plt.yticks(fontproperties=MyData.Efont_prop2, fontsize=8)
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=8)

#第二张图
plt.subplot(122)
plt.plot(X_tsne[y_pred==0, 0], X_tsne[y_pred==0, 1], linestyle='', marker='o', color=MyData.colors_rgb[0], markersize=1, label='Loser')
plt.plot(X_tsne[y_pred==1, 0], X_tsne[y_pred==1, 1], linestyle='', marker='o', color=MyData.colors_rgb[8], markersize=1, label='Winner')
plt.yticks(fontproperties=MyData.Efont_prop2, fontsize=8)
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=8)
plt.legend(prop=MyData.Efont_prop2)
plt.tight_layout()
plt.savefig("images/P3/X_testTSNE.png", format="png")

## Improved Markov Chain Model
### Stage 1 Forward propagation

In [32]:
alpha, cnt = 0.7, 0
Player_SVM = np.zeros([2, Members0.shape[1]-1])

for c, i in enumerate(MyData.data['match_id'].unique()):
    if c >= 0 and c <= 0:
        continue
    if c == 2:
        break

    A1, B1, A2, B2 = 0, 0, 0, 0
    Last_Vic = 0   # no victory
    Distance = [0, 0]
    Comp = MyData.data[MyData.data['match_id'] == i]
    M1, M2 = np.zeros([len(Comp)+1, 1]), np.zeros([len(Comp)+1, 1])
    M1[0], M2[0] = 0, 0
    SumAlpha, ItemAlpha = 0, 1
    ItemM1, ItemM2 = 0, 0

    Num1 = 2*(c+1) - 2
    Num2 = Num1 + 1

    Qa = np.zeros([len(Comp), 2])
    Qb = np.zeros([len(Comp), 2])
    pa = np.zeros([len(Comp), 1])
    pb = np.zeros([len(Comp), 1])
    for j in range(len(Comp)):

        # Point Prob Predict
        ItemM1 = ItemM1*(1-alpha) + M1[j]
        ItemM2 = ItemM2*(1-alpha) + M2[j]
    
        SumAlpha += ItemAlpha
        ItemAlpha*=(1-alpha)

        Player_SVM[0, :] = (1 + ItemM1 / SumAlpha) * np.array(Members0.iloc[Num1, :-1])
        Player_SVM[1, :] = (1 + ItemM2 / SumAlpha) * np.array(Members0.iloc[Num2, :-1])
        
        Qa[j, :] = best_model.predict_proba(Player_SVM[0, :].reshape((1, 15)))
        Qb[j, :] = best_model.predict_proba(Player_SVM[1, :].reshape((1, 15)))

        pa[j, 0] = max(Qa[j, :]) / (max(Qa[j, :]) + max(Qb[j, :]))
        pb[j, 0] = max(Qb[j, :]) / (max(Qa[j, :]) + max(Qb[j, :]))

        # State Trans
        if pa[j, 0] >= pb[j, 0]:
            Last_Vic = 1

        # Update the dynamic indicators
        A1 = A1 + Last_Vic if Last_Vic == 1 else 0
        A1 = 0 if A1 == 1 else A1

        B1 = B1 + 1 if Last_Vic == 2 else 0
        B1 = 0 if B1 == 1 else B1

        T11 = sum(Comp.iloc[:j+1, 39]) / Members0.iloc[Num1, -1]
        T12 = 1 if Comp.iloc[j, 13] == 1 else -1
        T13 = 1 if Comp.iloc[j, 35] == 1 else 0
        T14 = 1 if Comp.iloc[j, 27] == 1 else 0
        M1[j+1] = A1 * W[0] + B1 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]

        A2 = A2 + Last_Vic if Last_Vic == 2 else 0
        A2 = 0 if A2 == 1 else A2

        B2 = B2 + 1 if Last_Vic == 1 else 0
        B2 = 0 if B2 == 1 else B2
        
        T11 = sum(Comp.iloc[:j+1, 40]) / Members0.iloc[Num2, -1]
        T12 = 1 if Comp.iloc[j, 13] == 2 else -1
        T13 = 1 if Comp.iloc[j, 36] == 1 else 0
        T14 = 1 if Comp.iloc[j, 28] == 1 else 0

        M2[j+1] = A2 * W[0] + B2 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]


In [ ]:
print(pa[:6, :])
print(pb[:6, :])
print(sum(pa==pb))

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(12, 4), dpi=MyData.dpi)

plt.fill_between(range(len(pa)), y1=0.5, y2=pa[:, 0], color=MyData.colors_rgb[8], alpha=0.2, label='Prob1')
plt.fill_between(range(len(pb)), y1=0.5, y2=pb[:, 0], color=MyData.colors_rgb[1], alpha=0.2, label='Prob2')

plt.plot(range(len(pa)), pa, color=MyData.colors_rgb[8])


plt.plot(range(len(pb)), pb, color=MyData.colors_rgb[1])
plt.axhline(y=0.5, color='green', linestyle='--')


plt.yticks([0.3, 0.4, 0.5, 0.6, 0.8], ['0.3', '0.4', '0.5', '0.6', '0.8'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.ylabel('P(Point)', fontproperties=MyData.Efont_prop2, fontsize=10)
plt.legend(prop=MyData.Efont_prop2)
plt.margins(x=0)
plt.tight_layout()
plt.savefig("images/P3/P(Point)_1302.png", format="png")

In [ ]:
Last_Index = -1
for i in range(1, len(Sgp)):
    
    print(Sgp[-i], SgpIndex[-i], Last_Index)
    Last_Index = SgpIndex[-i]

### Stage 2 Backward propagation

In [35]:
# Predict the P result
Pa = np.zeros([len(Comp), 1])
Pb = np.zeros([len(Comp), 1])

UPA, UPB = 1, 0
Last_Index = -1
cnt = -1
for i in range(1, len(Sgp)):

    Pana = pa[SgpIndex[-i]:Last_Index, 0]
    Panb = pb[SgpIndex[-i]:Last_Index, 0]
    UltiP = {'(4,0)':UPA,'(4,1)':UPA,'(4,2)':UPA, '(0,4)':UPB, '(1,4)':UPB, '(2,4)':UPB, '(5,3)':UPA, '(3,5)':UPB}

    Pointa, Pointb = sum(Pana > Panb), sum(Pana < Panb)
    ccnt = 0
    for j in range(1, len(Pana) + 1):
        

        def ReturnV(Pa, Pb):
            if '('+str(Pa) +','+str(Pb)+')' in UltiP:
                return UltiP['('+str(Pa) +','+str(Pb)+')']
            elif Pa == 3 and Pb == 3:
                return Pana[-j+1] * Pana[-j+2]/(1 - Pana[-j+1] - Pana[-j+2] + 2 * Pana[-j+1] * Pana[-j+2])
            else:
                return Pana[-j+1] * ReturnV(Pa+1, Pb) + Panb[-j+1] * ReturnV(Pa, Pb+1)

        if Pointa >= 4 and (Pointa - Pointb) >= 2:
            Pa[cnt] = UPA
        elif Pointb >= 4 and (Pointb - Pointa) >= 2:
            Pa[cnt] = UPB
        elif Pointa == 3 and Pointb == 3:
            Pa[cnt] = Pana[-j+1] * Pana[-j+2]/(1 - Pana[-j+1] - Pana[-j+2] + 2 * Pana[-j+1] * Pana[-j+2])
        else:
            if Pointa > 3:
                Pia = Pointa - 3
            if Pointb > 3:
                Pib = Pointb - 3

            Pa[cnt] = ReturnV(Pia, Pib)

        Pb[cnt] = 1 - Pa[cnt]
        # print([Pointa, Pointb, Pa[cnt]])
        cnt -= 1
        ccnt += 1
        if Pana[-j] > Panb[-j]:
            Pointa -= 1
        else:
            Pointb -= 1
    UPA = 0.8
    UPB = 0.2
    # print([ccnt, Sgp[-i], SgpIndex[-i], Last_Index])
    Last_Index = SgpIndex[-i]

Pa[0], Pb[0] = 0.5, 0.5
Pa[-1], Pb[-1] = 1, 0

In [172]:
Sgp = []
SgpIndex = []
for i in range(len(Comp)):
    T = str(Comp.iloc[i, 4]) + '-' + str(Comp.iloc[i, 5])
    if i == 0:
        Sgp.append(T)
        SgpIndex.append(i)
    elif T == Sgp[-1]:
        continue
    Sgp.append(T)
    SgpIndex.append(i)

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(12, 4), dpi=MyData.dpi)

plt.fill_between(range(len(Pb)), y1=0.5, y2=Pb[:, 0], color=MyData.colors_rgb[1], alpha=0.2, label='Prob1')
plt.fill_between(range(len(Pa)), y1=0.5, y2=Pa[:, 0], color=MyData.colors_rgb[8], alpha=0.2, label='Prob2')
plt.plot(range(len(Pa)), Pa, color=MyData.colors_rgb[8])


plt.plot(range(len(Pb)), Pb, color=MyData.colors_rgb[1])

plt.axhline(y=0.5, color='green', linestyle='--')


plt.yticks([0.2, 0.4, 0.5, 0.6, 0.8, 1.0], ['0.2', '0.4', '0.5', '0.6', '0.8', '1.0'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.ylabel('P(Win)', fontproperties=MyData.Efont_prop2, fontsize=10)
plt.legend(prop=MyData.Efont_prop2)
plt.margins(x=0)
plt.tight_layout()
# plt.savefig("images/P3/P(Win)_1302.png", format="png")

In [ ]:
import shap
shap.initjs()

In [154]:
def model_predict_proba(X):
    return best_model.predict_proba(X)

In [ ]:
LabelName = ['AR', 'DFR', 'SSER', 'FSR', 'FSSR', 'SSSR', 'SER', 'SBR', 'FRSR', 'SRSR', 'RR', 'BR', 'SSCR', 'RSR', 'SCR']
DX_test = pd.DataFrame(X_test, columns=LabelName)
print(DX_test.head())

In [ ]:
# explain the model's predictions using SHAP
explainer = shap.Explainer(model_predict_proba, X_train)

# Calculate SHAP values for a sample
shap_values = explainer(DX_test)


In [ ]:
# Visualize the explanation for the first prediction
# shap.summary_plot(shap_values, X_test, feature_names=Members.columns[:-1])

# visualize the first prediction's explanation
plt.figure(figsize=(10, 8), dpi=600)

shap.plots.waterfall(shap_values[0, :, 1])


In [ ]:
plt.figure(dpi=400)
shap.summary_plot(shap_values[:, :, 0], DX_test)

In [ ]:
plt.figure(dpi=400)
shap.summary_plot(shap_values[:, :, 1], DX_test, plot_type="bar")

In [170]:
alpha, cnt = 0.7, 0
Player_SVM = np.zeros([2, Members0.shape[1]-1])

for c, i in enumerate(MyData.data['match_id'].unique()):
    if c >= 0 and c <= 0:
        continue
    if c == 2:
        break

    A1, B1, A2, B2 = 0, 0, 0, 0
    Last_Vic = 0   # no victory
    Distance = [0, 0]
    Comp = MyData.data[MyData.data['match_id'] == i]
    M1, M2 = np.zeros([len(Comp)+1, 1]), np.zeros([len(Comp)+1, 1])
    M1[0], M2[0] = 0, 0
    SumAlpha, ItemAlpha = 0, 1
    ItemM1, ItemM2 = 0, 0

    Num1 = 2*(c+1) - 2
    Num2 = Num1 + 1

    Qa = np.zeros([len(Comp), 2])
    Qb = np.zeros([len(Comp), 2])
    pa = np.zeros([len(Comp), 1])
    pb = np.zeros([len(Comp), 1])
    for j in range(len(Comp)):

        # Point Prob Predict
        ItemM1 = ItemM1*(1-alpha) + M1[j]
        ItemM2 = ItemM2*(1-alpha) + M2[j]
    
        SumAlpha += ItemAlpha
        ItemAlpha*=(1-alpha)

        Player_SVM[0, :] = (1 + ItemM1 / SumAlpha) * np.array(Members0.iloc[Num1, :-1])
        Player_SVM[1, :] = (1 + ItemM2 / SumAlpha) * np.array(Members0.iloc[Num2, :-1])
        
        Qa[j, :] = best_model.predict_proba(Player_SVM[0, :].reshape((1, 15)))
        Qb[j, :] = best_model.predict_proba(Player_SVM[1, :].reshape((1, 15)))

        pa[j, 0] = max(Qa[j, :]) / (max(Qa[j, :]) + max(Qb[j, :]))
        pb[j, 0] = max(Qb[j, :]) / (max(Qa[j, :]) + max(Qb[j, :]))

        # State Trans
        if pa[j, 0] >= pb[j, 0]:
            Last_Vic = 1

        # Update the dynamic indicators
        A1 = A1 + Last_Vic if Last_Vic == 1 else 0
        A1 = 0 if A1 == 1 else A1

        B1 = B1 + 1 if Last_Vic == 2 else 0
        B1 = 0 if B1 == 1 else B1

        T11 = sum(Comp.iloc[:j+1, 39]) / Members0.iloc[Num1, -1]
        T12 = 1 if Comp.iloc[j, 13] == 1 else -1
        T13 = 1 if Comp.iloc[j, 35] == 1 else 0
        T14 = 1 if Comp.iloc[j, 27] == 1 else 0
        M1[j+1] = A1 * W[0] + B1 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]

        A2 = A2 + Last_Vic if Last_Vic == 2 else 0
        A2 = 0 if A2 == 1 else A2

        B2 = B2 + 1 if Last_Vic == 1 else 0
        B2 = 0 if B2 == 1 else B2
        
        T11 = sum(Comp.iloc[:j+1, 40]) / Members0.iloc[Num2, -1]
        T12 = 1 if Comp.iloc[j, 13] == 2 else -1
        T13 = 1 if Comp.iloc[j, 36] == 1 else 0
        T14 = 1 if Comp.iloc[j, 28] == 1 else 0

        M2[j+1] = A2 * W[0] + B2 * W[1] + T11 * W[2] + T12 * W[3] + T13 * W[4] + T14 * W[5]


In [ ]:

plt.fill_between(range(len(Pb)), y1=0.5, y2=Pb[:, 0], color=MyData.colors_rgb[1], alpha=0.2, label='Prob1')
plt.fill_between(range(len(Pa)), y1=0.5, y2=Pa[:, 0], color=MyData.colors_rgb[8], alpha=0.2, label='Prob2')
plt.plot(range(len(Pa)), Pa, color=MyData.colors_rgb[8])


plt.plot(range(len(Pb)), Pb, color=MyData.colors_rgb[1])

plt.axhline(y=0.5, color='green', linestyle='--')


plt.yticks([0.2, 0.4, 0.5, 0.6, 0.8, 1.0], ['0.2', '0.4', '0.5', '0.6', '0.8', '1.0'], fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(SgpIndex, Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)
plt.ylabel('P(Win)', fontproperties=MyData.Efont_prop2, fontsize=10)
plt.legend(prop=MyData.Efont_prop2)
plt.margins(x=0)
plt.tight_layout()
plt.savefig("images/P3/P(Win)_1302.png", format="png")

In [ ]:
sns.set(style="whitegrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(10, 8), dpi=MyData.dpi)
categories = ['Accuracy', 'Precision', 'Recall']

num_y = np.arange(len(MID))
height = 0.3

#计算每个图形高度的起始位置
movie1_start_y = num_y              
movie2_start_y = num_y + height      
movie3_start_y = num_y + 2 * height  

# 画水平柱状图
plt.barh(movie1_start_y, Acc, height=height)
plt.barh(movie2_start_y, Precision,  height=height)
plt.barh(movie3_start_y, Recall, height=height)

# 添加轴标签和标题
plt.yticks(num_y + height, MID, fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=9)
plt.show()


In [346]:
sns.set(style="whitegrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(8, 6), dpi=MyData.dpi)
categories = ['Accuracy', 'Precision', 'Recall']

height = 0.6

#计算每个图形高度的起始位置

# 画水平柱状图
plt.barh(MID, Recall, height=height, color=MyData.colors_rgb[6])

# 添加轴标签和标题
plt.yticks(MID, fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=11)
plt.savefig("images/P4/Accuracy.png", format="png")

In [347]:
sns.set(style="whitegrid")
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(8, 6), dpi=MyData.dpi)
categories = ['Accuracy', 'Precision', 'Recall']

height = 0.6

#计算每个图形高度的起始位置

# 画水平柱状图
plt.barh(MID, Acc, height=height, color=MyData.colors_rgb[9])

# 添加轴标签和标题
plt.yticks(MID, fontproperties=MyData.Efont_prop2, fontsize=10)
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=11)
plt.savefig("images/P4/Recall.png", format="png")

In [173]:
PointV = Comp['point_victor'].copy()
GameV = Comp['game_victor'].copy()
SetV = Comp['set_victor'].copy()

In [174]:
SetV[SetV == 1] = 0
SetV[SetV == 2] = 1

In [176]:
CSuSetV = SetV.cumsum()
CSuSetV -= 1
CSuSetV[CSuSetV == -1] = 0

In [ ]:

sns.set(style="whitegrid", rc={"grid.linewidth": 0.6, "grid.linestyle": "--"})
plt.rcParams['axes.unicode_minus'] = False
fig, ax1 = plt.subplots(figsize=(10, 2), dpi=MyData.dpi)

# Plot the first y-axis
ax1.plot(range(len(Pa)), Pa, color=MyData.colors_rgb[8])
ax1.fill_between(range(len(Pa)), y1=0.5, y2=Pa[:, 0], color=MyData.colors_rgb[8], alpha=0.2, label='Pw')
ax1.axhline(y=0.5, color='green', linestyle='--')
ax1.set_ylabel('P(Win)', fontproperties=MyData.Efont_prop2, fontsize=10)

# Set y-ticks for the first y-axis
ax1.set_yticks([0.2, 0.4, 0.5, 0.6, 0.8, 1.0])
ax1.set_yticklabels(['0.2', '0.4', '0.5', '0.6', '0.8', '1.0'], fontproperties=MyData.Efont_prop2, fontsize=10)

# Create a second y-axis
ax2 = ax1.twinx()
#ax2.plot(range(len(Pa)), PointV, '*g', markersize=4)
#ax2.plot(range(len(Pa)), GameV, '*r', markersize=4)
ax2.plot(range(len(Pa)), CSuSetV, 'orange', markersize=4, label='Ps')

ax2.set_ylim([-1, 3])
ax2.set_yticks([-1, 0, 1, 2, 3])
ax2.set_yticklabels(['-1', '0', '1', '2', '3'], fontproperties=MyData.Efont_prop2, fontsize=10)
ax2.set_ylabel('Set Victory', fontproperties=MyData.Efont_prop2, fontsize=10)

ax1.set_xticks(SgpIndex)
ax1.set_xticklabels(Sgp, fontproperties=MyData.Efont_prop2, fontsize=9, rotation=-90)

# Set layout and save the figure
plt.tight_layout()
plt.savefig("images/P4/CompareWithSet.png", format="png")




In [ ]:
PointV[PointV == 1] = 0
PointV[PointV == 2] = 1
print(PointV.cumsum())
print(PointV.sum())


In [ ]:
GameV[GameV == 1] = 0
GameV[GameV == 2] = 1
print(GameV.cumsum())
print(GameV.sum())

In [ ]:
SetV[SetV == 1] = 0
SetV[SetV == 2] = 1
print(SetV.cumsum())
print(SetV.sum())

In [190]:
MMAcc = np.zeros(Acc.shape)
for i in range(Acc.shape[0]):
    Mid = Acc.shape[1]/2
    for j in range(Acc.shape[1]):
        if j <= Mid:
            MMAcc[i, j] = MAcc[i]-j*0.005
        else:
            MMAcc[i, j] = MAcc[i]+(j-Mid)*0.005

In [ ]:
plt.figure(figsize=(10, 4), dpi=600)
for i in range(Acc.shape[1]):
    label = r'$\alpha$={:.0f}%'.format(i + 2)
    plt.scatter(range(len(MAcc)), MMAcc[:, i], s=4, c=MyData.colors_rgb[i], marker='o', alpha=0.5, label=label)

plt.plot(MAcc, linewidth=1.5)
plt.fill_between(range(len(MAcc)), MMAcc.min(axis=1), MMAcc.max(axis=1), alpha=0.2)
plt.yticks(fontproperties=MyData.Efont_prop2, fontsize=7)
plt.xticks(range(len(MID)), MID)
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=7, rotation=-90)
plt.ylim([0.6, 0.85])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=8)
plt.tight_layout()
plt.savefig('images/SR/SensitivityAnalysis.png', dpi=600)

In [180]:
MMAcc = np.zeros(Acc.shape)
for i in range(Acc.shape[0]):
    MMAcc[i, :] = (Acc[i, :] - Acc[i, :].min()) / (Acc[i, :].min())

In [139]:
STDAcc = (Acc - Acc.mean()) / Acc.std()
STDAcc = 0.2 * (STDAcc - STDAcc.min()) / STDAcc.max() - 0.1

In [ ]:
plt.figure(figsize=(10, 2), dpi=600)
for i in range(8):
    sns.kdeplot(np.array(STDAcc[:, i]), shade=True, label=r'$\beta$='+str(i+5)+'%')
plt.ylabel('')
plt.xticks(fontproperties=MyData.Efont_prop2, fontsize=7)
plt.yticks(fontproperties=MyData.Efont_prop2, fontsize=7)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=8)
plt.tight_layout()
plt.savefig('images/SR/RobustnessAnalysis.png', dpi=600)